# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [ ]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [ ]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k_values = list(range(1, 12))


In [ ]:
# Create an empy list to store the inertia values
inertia_values = []

In [ ]:
# Create a for loop to compute the inertia with each possible value of k
for k in range(1, 12):
    # Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=k, random_state=42)
    
    # Fit the model to the data using df_market_data_scaled
    model.fit(df_market_data_scaled)
    
    # Append the model.inertia_ to the inertia list
    inertia_values.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k_values': k_values,
    'inertia_values': inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

print(elbow_df)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(x='k_values', y='inertia_values', title='Elbow Curve to Find Optimal k', xlabel='Number of Clusters (k)', ylabel='Inertia', line_color='orange', line_width=3)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:**  3

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [ ]:
# Initialize the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters=3, random_state=42)

In [ ]:
# Fit the K-Means model using the scaled data
kmeans_model.fit(df_market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters_scaled = kmeans_model.predict(df_market_data_scaled)

# View the resulting array of cluster values.
print(clusters_scaled)

In [ ]:
# Create a copy of the DataFrame
df_market_data_scaled_clusters = df_market_data_scaled.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled_clusters['clusters'] = clusters_scaled

# Display sample data
df_market_data_scaled_clusters.head()


In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
# Create a scatter plot using hvPlot
scatter_plot_scaled = df_market_data_scaled_clusters.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters (Scaled Data)",
    xlabel="24h Price Change Percentage",
    ylabel="7d Price Change Percentage",
    cmap='Category10',  # Choose a color map for the clusters
    legend='top_left'   # Adjust the legend position
)

# Show the scatter plot
scatter_plot_scaled


---

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_data = pca.fit_transform(df_market_data_scaled)


# View the first five rows of the DataFrame. 
print(pd.DataFrame(pca_data, columns=["PC1", "PC2", "PC3"]).head())

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_

# Display the explained variance for each principal component
print("Explained Variance for PC1, PC2, and PC3:", explained_variance)

In [ ]:
# Calculate the total explained variance of the three principal components
total_explained_variance = sum(explained_variance)

# Display the total explained variance
print("Total Explained Variance of the Three Principal Components:", total_explained_variance)


#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.8950316570309841

In [ ]:
# Create a new DataFrame with the PCA data
df_pca = pd.DataFrame(pca_data, columns=["PC1", "PC2", "PC3"], index=df_market_data_scaled.index)


In [ ]:
# Copy the crypto names from the original data
df_pca["coin_id"] = df_market_data_scaled.index

# Set the coin_id column as the index
df_pca = df_pca.set_index("coin_id")

# Display sample data
df_pca.head()

---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [ ]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k_values_pca = list(range(1, 12))

In [ ]:
# Create an empy list to store the inertia values
inertia_values_pca = []

In [ ]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k_pca in k_values_pca:
    # Initialize KMeans model with the current k value
    model_pca = KMeans(n_clusters=k_pca, random_state=42)
    
    # Fit the model to the PCA data
    model_pca.fit(df_pca)
    
    # Append the model.inertia_ to the inertia list
    inertia_values_pca.append(model_pca.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve for PCA data
elbow_data_pca = {
    'k_values_pca': k_values_pca,
    'inertia_values_pca': inertia_values_pca
}

# Create a DataFrame with the data to plot the Elbow curve for PCA data
elbow_df_pca = pd.DataFrame(elbow_data_pca)

# Display the DataFrame for PCA data
print(elbow_df_pca)


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df_pca.hvplot.line(x='k_values_pca', y='inertia_values_pca', title='Elbow Curve to Find Optimal k (PCA Data)', xlabel='Number of Clusters (k)', ylabel='Inertia', line_color='orange', line_width=3)


#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 3


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** no

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [ ]:
# Initialize the K-Means model using the best value for k
kmeans_model_pca = KMeans(n_clusters=3, random_state=42)

In [ ]:
# Fit the K-Means model using the PCA data
kmeans_model_pca.fit(df_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters_pca = kmeans_model_pca.predict(df_pca)

# View the resulting array of cluster values
print(clusters_pca)

In [ ]:
# Create a copy of the DataFrame with the PCA data
df_pca_clusters = df_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_pca_clusters['clusters'] = clusters_pca

# Display sample data
df_pca_clusters.head()


In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
# Create a scatter plot using hvPlot
scatter_plot_pca = df_pca_clusters.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="clusters",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters (PCA Data)",
    xlabel="Principal Component 1 (PC1)",
    ylabel="Principal Component 2 (PC2)",
    cmap='Category10',  # Choose a color map for the clusters
    legend='top_left'   # Adjust the legend position
)

# Show the scatter plot
scatter_plot_pca

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [ ]:
# Create a composite plot for the Elbow Curves
elbow_curve_original = elbow_df.hvplot.line(x='k_values', y='inertia_values', title='Elbow Curve (Original Data)', xlabel='Number of Clusters (k)', ylabel='Inertia')
elbow_curve_pca = elbow_df_pca.hvplot.line(x='k_values_pca', y='inertia_values_pca', title='Elbow Curve (PCA Data)', xlabel='Number of Clusters (k)', ylabel='Inertia')
composite_elbow_curves = elbow_curve_original + elbow_curve_pca

# Display the composite plot for Elbow Curves
composite_elbow_curves


In [ ]:
# Create a composite plot for the Cryptocurrencies Clusters
scatter_plot_original =  df_market_data_scaled_clusters.hvplot.scatter(x='price_change_percentage_24h', y='price_change_percentage_7d', by='clusters', hover_cols=['coin_id'], title='Cryptocurrency Clusters (Original Data)', xlabel='Price Change Percentage (24h)', ylabel='Price Change Percentage (7d)', cmap='Category10', legend='top_left')
scatter_plot_pca = df_pca_clusters.hvplot.scatter(x='PC1', y='PC2', by='clusters', hover_cols=['coin_id'], title='Cryptocurrency Clusters (PCA Data)', xlabel='Principal Component 1 (PC1)', ylabel='Principal Component 2 (PC2)', cmap='Category10', legend='top_left')
composite_scatter_plots = scatter_plot_original + scatter_plot_pca

# Display the composite plot for Cryptocurrency Clusters
composite_scatter_plots


#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Reducing the number of features in K-Means clustering may simplify and improve computational efficiency, but it risks losing relevant information, impacting cluster interpretability, and necessitates careful consideration of feature selection based on domain knowledge to ensure meaningful results.